# The ML Pipeline for a Model to Predict Length of Hospital Delivery Stay

In [1]:
import pandas as pd
import numpy as np
import math
from matplotlib import pyplot as plt
import matplotlib

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import PredefinedSplit

from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler

from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.compose import ColumnTransformer

from sklearn.linear_model import Lasso
from sklearn.neighbors import KNeighborsRegressor

### Gather Data

In [3]:
# specifying data types for the columns to maintain formatting from original data
data_types = {
    'hospital_service_area': object, 
    'hospital_county': object,
    'operating_certificate_number': object, 
    'permanent_facility_id': object,
    'facility_name': object, 
    'age_group': object, 
    'zip_code_3_digits': object, 
    'gender': object, 
    'race': object,
    'ethnicity': object, 
    'payment_typology_1': object, 
    'payment_typology_2': object,
    'payment_typology_3': object, 
    'length_of_stay': int
}

In [4]:
all_visits = pd.read_csv('../data/planned_deliveries.csv', dtype=data_types)
all_visits = all_visits.loc[:, all_visits.columns != 'Unnamed: 0']
y = all_visits['length_of_stay']
X = all_visits.loc[:, all_visits.columns != 'length_of_stay']

### Compute Baseline Score

In [13]:
mean_length_of_stay = int(np.around(np.mean(y), 0))
median_length_of_stay = int(np.around(np.median(y), 0))
print('Mean length of stay:', mean_length_of_stay)
print('Median length of stay:', median_length_of_stay)

Mean length of stay: 2
Median length of stay: 2


In [15]:
y_pred_mean = pd.Series([2]*len(y))

#### RMSE [days]

In [17]:
baseline_rmse = mean_squared_error(y, y_pred_mean, squared=False)
print('Baseline RMSE [days]:', baseline_rmse)

Baseline RMSE [days]: 0.9521077950265668


#### R^2 [dimensionless]  
**Pretty sure this is unnecessary to do though since by definition R^2 should = 0 for the expected (average) guess of y**

In [21]:
baseline_r2 = r2_score(y, y_pred_mean)
print('Baseline R^2 [dimensionless]:', baseline_r2)

Baseline R^2 [dimensionless]: -0.11501180761222085


### Split, Train, and Cross Validate

In [24]:
def stratified_continuous_split(X:pd.DataFrame, y:pd.Series, train_size:float, val_size:float, test_size:float, random_state:int):
    '''
    Performs a stratified split of inputted data (with respect to y) into a training set, validation set, and test set to specified percentages 
    of the data using verstack's scsplit and performs basic error checking.

    Parameters:
    - X: a 2D pandas DataFrame, the feature matrix
    - y: a 1D pandas Series, the target variable matrix matching X
    - train_size: a float between 0 and 1, the percentage of X which should be training data
    - val_size: a float between 0 and 1, the percentage of X which should be reserved for validation
    - test_size: a float between 0 and 1, the percentage of X which should be reserved for final testing
    - random_state: an int, the random state to split with
    Note: The sum of train_size + val_size + test_size must be 1.0 (100% of X).

    Returns:
    - (X_train) a 2D pandas DataFrame, the feature matrix of training data
    - (y_train) a 1D pandas Series, the target variable matrix for training data
    - (X_val) a 2D pandas DataFrame, the feature matrix of validation data
    - (y_val) a 1D pandas Series, the target variable matrix for validation data
    - (X_test) a 2D pandas DataFrame, the feature matrix of testing data
    - (y_test) a 1D pandas Series, the target variable matrix for testing data

    Raises:
    - ValueError for invalid input
    '''
    from verstack.stratified_continuous_split import scsplit
    
    if ((train_size + val_size + test_size) != 1):
        raise ValueError('Your train_size + val_size + test_size must add up to 1 (100%)!')
    if (not isinstance(random_state, int)):
        raise ValueError('Your random_state must be an int!')

    X_train, X_other, y_train, y_other = scsplit(X, y, stratify=y, test_size=(1-train_size), random_state=random_state)
    
    X_len = X.shape[0]
    test_percent_of_other = (test_size * X_len)/(X_len - (train_size * X_len))
    X_other = X_other.reset_index(drop=True)
    y_other = y_other.reset_index(drop=True)
    
    X_val, X_test, y_val, y_test = scsplit(X_other, y_other, stratify=y_other, test_size=test_percent_of_other, random_state=random_state)

    # basic error checking to check that split returned train, val, and test of expected sizes
    train_count_low = (int)(train_size * X_len)
    train_count_high = ceil(train_size * X_len)
    val_count_low = (int)(val_size * X_len)
    val_count_high = ceil(val_size * X_len)
    test_count_low = (int)(test_size * X_len)
    test_count_high = ceil(test_size * X_len)
    
    Xtrain_fin = X_train.shape[0]
    ytrain_fin = y_train.shape[0]
    Xval_fin = X_val.shape[0]
    yval_fin = y_val.shape[0]
    Xtest_fin = X_test.shape[0]
    ytest_fin = y_test.shape[0]
    
    if not (((Xtrain_fin == train_count_low) or (Xtrain_fin == train_count_high)) and ((ytrain_fin == train_count_low) or (ytrain_fin == train_count_high))):
        raise ValueError(f'Training set size should be approx. {train_size * X_len}, instead is: {X_train.shape[0]}')
    if not (((Xval_fin == val_count_low) or (Xval_fin == val_count_high)) and ((yval_fin == val_count_low) or (yval_fin == val_count_high))):
        raise ValueError(f'Validation set size should be approx. {val_size * X_len}, instead is: {X_val.shape[0]}')
    if not (((Xtest_fin == test_count_low) or (Xtest_fin == test_count_high)) and ((ytest_fin == test_count_low) or (ytest_fin == test_count_high))):
        raise ValueError(f'Test set size should be approx. {test_size * X_len}, instead is: {X_test.shape[0]}')

    return X_train, y_train, X_val, y_val, X_test, y_test

In [25]:
# def stratified_continuous_split(X:pd.DataFrame, y:pd.Series, train_size:float, val_size:float, test_size:float, random_state:int):
#     '''
#     Performs a stratified split of inputted data (with respect to y) into 2 sets of specified percentages 
#     of the data using verstack's scsplit and performs basic error checking.

#     Parameters:
#     - X: a 2D pandas DataFrame, the feature matrix
#     - y: a 1D pandas Series, the target variable matrix matching X
#     - train_size: a float between 0 and 1, the percentage of X which should be training data
#     - test_size: a float between 0 and 1, the percentage of X which should be reserved for testing
#     - random_state: an int, the random state to split with
#     Note: The sum of train_size + test_size must be 1.0 (100% of X).

#     Returns:
#     - (X_train) a 2D pandas DataFrame, the feature matrix of training data
#     - (y_train) a 1D pandas Series, the target variable matrix for training data
#     - (X_test) a 2D pandas DataFrame, the feature matrix of testing data
#     - (y_test) a 1D pandas Series, the target variable matrix for testing data

#     Raises:
#     - ValueError for invalid input
#     '''
#     from verstack.stratified_continuous_split import scsplit
    
#     if ((train_size + val_size + test_size) != 1):
#         raise ValueError('Your train_size + test_size must add up to 1 (100%)!')
#     if (not isinstance(random_state, int)):
#         raise ValueError('Your random_state must be an int!')

#     X_train, X_other, y_train, y_other = scsplit(X, y, stratify=y, test_size=test_size, random_state=random_state)

#     # basic error checking to check that split returned train and test of expected sizes
#     train_count_low = (int)(train_size * X_len)
#     train_count_high = ceil(train_size * X_len)
#     test_count_low = (int)(test_size * X_len)
#     test_count_high = ceil(test_size * X_len)
    
#     Xtrain_fin = X_train.shape[0]
#     ytrain_fin = y_train.shape[0]
#     Xtest_fin = X_test.shape[0]
#     ytest_fin = y_test.shape[0]
    
#     if not (((Xtrain_fin == train_count_low) or (Xtrain_fin == train_count_high)) and ((ytrain_fin == train_count_low) or (ytrain_fin == train_count_high))):
#         raise ValueError(f'Training set size should be approx. {train_size * X_len}, instead is: {X_train.shape[0]}')
#     if not (((Xtest_fin == test_count_low) or (Xtest_fin == test_count_high)) and ((ytest_fin == test_count_low) or (ytest_fin == test_count_high))):
#         raise ValueError(f'Test set size should be approx. {test_size * X_len}, instead is: {X_test.shape[0]}')

#     return X_train, y_train, X_test, y_test

In [ ]:
def MLpipe_Stratified_Continous_RMSE(X, y, preprocessor, ML_algo, param_grid):
    '''
    This function splits the data to train, validation, and test (60/20/20).
    The RMSE is minimized in cross-validation.
    
    This function:
    1. Loops through 10 different random states
    2. Splits the data 60/20/20.
    3. Fits a model using GridSearchCV with the predefined Preprocessor 
    4. Calculates the model's error on the test set 
    5. Returns a list of 10 test scores and 10 best models
    '''
    
    # lists to be returned
    test_scores = []
    best_models = []

    nr_states = 10
    for i in range(nr_states):
        rs = 28 * i
        print('Random State:', rs) # TEMPORARY

        # split
        X_train, y_train, X_val, y_val, X_test, y_test = stratified_continous_split(X, y, train_size=0.6, val_size=0.2, test_size=0.2, random_state=rs)
        
        # preprocess, train, and perform cross-validation
        pipe = make_pipeline(preprocessor, ML_algo)
        grid = GridSearchCV(pipe, param_grid=param_grid, scoring='neg_mean_squared_error', # using PredefinedSplit??????
                            cv=kf, return_train_score=True, n_jobs=-1, verbose=True)
        grid.fit(X_other, y_other)
        results = pd.DataFrame(grid.cv_results_)
        #print(results) # TEMPORARY

        # save results
        print('    Best Model Parameters:', grid.best_params_) # TEMPORARY
        print('    Validation RMSE Score:', grid.best_score_) # TEMPORARY
        best_models.append(grid)
        y_test_pred = grid.predict(X_test)
        test_score = mean_squared_error(y_test, y_test_pred, squared=False)
        test_scores.append(test_score)
        print('    Test RMSE Score:', test_score) # TEMPORARY
        
    return test_scores, best_models

In [26]:
help(GridSearchCV)

Help on class GridSearchCV in module sklearn.model_selection._search:

class GridSearchCV(BaseSearchCV)
 |  GridSearchCV(estimator, param_grid, *, scoring=None, n_jobs=None, refit=True, cv=None, verbose=0, pre_dispatch='2*n_jobs', error_score=nan, return_train_score=False)
 |  
 |  Exhaustive search over specified parameter values for an estimator.
 |  
 |  Important members are fit, predict.
 |  
 |  GridSearchCV implements a "fit" and a "score" method.
 |  It also implements "score_samples", "predict", "predict_proba",
 |  "decision_function", "transform" and "inverse_transform" if they are
 |  implemented in the estimator used.
 |  
 |  The parameters of the estimator used to apply these methods are optimized
 |  by cross-validated grid-search over a parameter grid.
 |  
 |  Read more in the :ref:`User Guide <grid_search>`.
 |  
 |  Parameters
 |  ----------
 |  estimator : estimator object
 |      This is assumed to implement the scikit-learn estimator interface.
 |      Either est

In [2]:
help(PredefinedSplit)

Help on class PredefinedSplit in module sklearn.model_selection._split:

class PredefinedSplit(BaseCrossValidator)
 |  PredefinedSplit(test_fold)
 |  
 |  Predefined split cross-validator
 |  
 |  Provides train/test indices to split data into train/test sets using a
 |  predefined scheme specified by the user with the ``test_fold`` parameter.
 |  
 |  Read more in the :ref:`User Guide <predefined_split>`.
 |  
 |  .. versionadded:: 0.16
 |  
 |  Parameters
 |  ----------
 |  test_fold : array-like of shape (n_samples,)
 |      The entry ``test_fold[i]`` represents the index of the test set that
 |      sample ``i`` belongs to. It is possible to exclude sample ``i`` from
 |      any test set (i.e. include sample ``i`` in every training set) by
 |      setting ``test_fold[i]`` equal to -1.
 |  
 |  Examples
 |  --------
 |  >>> import numpy as np
 |  >>> from sklearn.model_selection import PredefinedSplit
 |  >>> X = np.array([[1, 2], [3, 4], [1, 2], [3, 4]])
 |  >>> y = np.array([0, 0,